# 6. PREDICTING AIBNB RENTAL PRICE IN CHICAGO
---

## 1. Introducing the Data

Source:
- [Inside AirBnB](http://insideairbnb.com/get-the-data.html)
- Date: 08/30/2020 (`last scraped`)
- Source data file description: `Detailed Listings Data for Chicago`
- Source file name: `listings.csv.gz`
- Chicago, Illinois, United States

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", 99)
pd.set_option("display.max_rows", 999)
pd.set_option('precision', 3)

chicago = pd.read_csv('data/chicago_airbnb.gz')
print(chicago.shape)
chicago.head()

(6550, 74)


,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2384,https://www.airbnb.com/rooms/2384,20200830040957,2020-08-31,"Hyde Park - Walk to UChicago, 10 min to McCormick",Visiting The University of Chicago or attendin...,My apartment is one block from beautiful Jacks...,https://a0.muscache.com/pictures/acf6b3c0-47f2...,2613,https://www.airbnb.com/users/show/2613,Rebecca,2008-08-29,"Chicago, Illinois, United States",My 2 bdrm apartment is a 2nd floor walk-up in ...,within a few hours,100%,100%,t,https://a0.muscache.com/im/pictures/user/8bb47...,https://a0.muscache.com/im/pictures/user/8bb47...,Hyde Park,1,1,"['email', 'phone', 'reviews', 'manual_offline'...",t,t,"Chicago, Illinois, United States",Hyde Park,NaN,41.788,-87.588,Private room in condominium,Private room,1,NaN,1 shared bath,1.0,1.0,"[""Hangers"", ""Extra pillows and blankets"", ""Wif...",$66.00,2,90,2,2,1125,1125,2.0,1125.0,NaN,t,29,59,89,363,2020-08-31,178,11,0,2015-01-09,2019-12-15,100.0,10.0,10.0,10.0,10.0,10.0,10.0,R17000015609,f,1,0,1,0,2.59
1,4505,https://www.airbnb.com/rooms/4505,20200830040957,2020-08-30,394 Great Reviews. 127 y/o House. 40 yds to tr...,We have been closed for quite a while and will...,NaN,https://a0.muscache.com/pictures/10628083/3543...,5775,https://www.airbnb.com/users/show/5775,Craig & Kathleen,2008-12-29,"Chicago, Illinois, United States",Incredibly creative(Kathleen) and stunningly a...,within an hour,100%,96%,t,https://a0.muscache.com/im/users/5775/profile_...,https://a0.muscache.com/im/users/5775/profile_...,Little Village,1,1,"['email', 'phone', 'reviews', 'kba']",t,t,NaN,South Lawndale,NaN,41.855,-87.697,Entire apartment,Entire home/apt,7,NaN,1 bath,3.0,5.0,"[""Hangers"", ""Extra pillows and blankets"", ""Wif...",$94.00,2,60,2,2,60,60,2.0,60.0,NaN,t,0,0,0,145,2020-08-30,395,14,0,2009-03-06,2020-07-14,96.0,10.0,9.0,10.0,10.0,10.0,9.0,R19000037538,t,1,1,0,0,2.82
2,7126,https://www.airbnb.com/rooms/7126,20200830040957,2020-08-30,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,"Ukrainian Village was just named ""Hottest Neig...",https://a0.muscache.com/pictures/51073/16c81c7...,17928,https://www.airbnb.com/users/show/17928,Sarah,2009-05-19,"Chicago, Illinois, United States",We live in Chicago. We love to travel and see ...,within an hour,100%,98%,t,https://a0.muscache.com/im/users/17928/profile...,https://a0.muscache.com/im/users/17928/profile...,Ukrainian Village,2,2,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Chicago, Illinois, United States",West Town,NaN,41.903,-87.682,Entire apartment,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Hangers"", ""Bathtub"", ""Wifi"", ""TV"", ""Extra pi...",$61.00,2,60,2,2,1125,1125,2.0,1125.0,NaN,t,9,9,33,308,2020-0

The dataset has 74 columns some of which we will have to get rid of because we don't think they are relevant to our prediction model design. Here is an example of some of the deleted columns:
- `id`
- `listing_url`
- `scrape_id`
- `last_scraped`
- `name`
- and more

## 2. Selecting Relevant Columns
The number of columns was reduced down from 74 to 28. Further cleaning is in order because some of the columns can't be used as they are. For example:
- `bathrooms`: is just `NaN` values. But we can use `bathrooms_text` to fill in the values and delete the text column
- `amenities`: is a column of lists. There are more than 5,000 different lists in this column but, because each list is made of a combination of sets of already-define keywords, we can do something good with the information. For example we can create an `amenity_score` column and calculate the score of each listing based on the keywords in the list

In [2]:
cols_1 = ['host_response_time', 'host_response_rate', 'host_acceptance_rate',
          'host_is_superhost', 'neighbourhood_cleansed', 'latitude', 'longitude', 
          'property_type', 'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 
          'bedrooms', 'beds', 'amenities', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 
          'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 
          'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 
          'review_scores_value', 'instant_bookable', 'reviews_per_month', 'price'
         ]

In [3]:
chicago_1 = chicago[cols_1]
print(chicago_1.shape)
chicago_1.head()

(6550, 28)


,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,price
0,within a few hours,100%,100%,t,Hyde Park,41.788,-87.588,Private room in condominium,Private room,1,NaN,1 shared bath,1.0,1.0,"[""Hangers"", ""Extra pillows and blankets"", ""Wif...",2,90,178,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,2.59,$66.00
1,within an hour,100%,96%,t,South Lawndale,41.855,-87.697,Entire apartment,Entire home/apt,7,NaN,1 bath,3.0,5.0,"[""Hangers"", ""Extra pillows and blankets"", ""Wif...",2,60,395,96.0,10.0,9.0,10.0,10.0,10.0,9.0,t,2.82,$94.00
2,within an hour,100%,98%,t,West Town,41.903,-87.682,Entire apartment,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Hangers"", ""Bathtub"", ""Wifi"", ""TV"", ""Extra pi...",2,60,384,93.0,10.0,9.0,10.0,10.0,10.0,10.0,f,2.83,$61.00
3,within an hour,92%,97%,t,Lincoln Park,41.918,-87.638,Entire apartment,Entire home/apt,4,NaN,1 bath,1.0,0.0,"[""Hangers"", ""Wifi"", ""TV"", ""Smoke alarm"", ""Carb...",4,180,49,92.0,10.0,9.0,10.0,9.0,10.0,9.0,t,0.63,$67.00
4,within a few hours,90%,95%,f,Hyde Park,41.796,-87.593,Private room in apartment,Private room,2,NaN,1 shared bath,1.0,1.0,"[""Hangers"", ""Extra pillows and blankets"", ""Wif...",1,1125,44,82.0,8.0,7.0,9.0,9.0,10.0,9.0,f,0.62,$22.00


## 3. Encoding `host_response_time`

In [4]:
chicago_1['host_response_time'].value_counts(dropna=False)

within an hour        3622
NaN                   1384
within a few hours     798
within a day           579
a few days or more     167
Name: host_response_time, dtype: int64

We can change the category names to terms that are short and informative. We could, of course, turn this into a numerical column whose values are expressed in hours but that wouldn't help more than just keeping the categories we have. Here are the new categories we will make:
- `1H`: within an hour
- `3H`: within a few hours
- `24H`: within a day
- `72H`: a few days or more
- `Missing`: NaN

In [5]:
chicago_2 = chicago_1.copy()
host_rt_map = {'within an hour': '1H',     
               'within a few hours': '3H',
               'within a day': '24H',
               'a few days or more': '72H',
               np.nan: 'Missing'
              }              
chicago_2['host_rt'] = [host_rt_map[item] for item in chicago_2['host_response_time']]
chicago_2['host_rt'].value_counts() / len(chicago_2)

1H         0.553
Missing    0.211
3H         0.122
24H        0.088
72H        0.025
Name: host_rt, dtype: float64

In [6]:
cols_2 = ['host_rt']

We can see that the `host_rt`, or host response time, is now in 5 categories. The null values had to be included in their own category since they account for over 20% of the entries and, therefore, you can't simply impute the mode. That would alter the information in a big way. Finally, we are starting a new column list, `cols_2` for the `chicago_2` dataframe. The list will grow and, eventually, its contents will end up being the columns for `chicago_2`

## 4. Removing the % Sign
Next, let's remove the % sign from `host_response_rate` and `host_acceptance_rate`.

In [7]:
chicago_2['host_rr'] = chicago_2['host_response_rate'].str.replace('%', '')
chicago_2['host_rr'] = chicago_2['host_rr'].astype('float')
chicago_2['host_rr'].value_counts(dropna=False).head(10)

100.0    3631
NaN      1384
90.0      159
99.0      144
96.0      126
86.0      126
0.0        94
98.0       91
97.0       77
80.0       66
Name: host_rr, dtype: int64

In [8]:
chicago_2['host_ar'] = chicago_2['host_acceptance_rate'].str.replace('%', '')
chicago_2['host_ar'] = chicago_2['host_ar'].astype('float')
chicago_2['host_ar'].value_counts(dropna=False).head(10)

100.0    1763
99.0      689
NaN       615
98.0      339
97.0      239
95.0      233
96.0      222
94.0      178
86.0      159
93.0      131
Name: host_ar, dtype: int64

With that done, we are left with a problem of a lot of missing values. We will take care of that in `Feature Engineering` by either imputing a value or by creating a missing category, if we decide to bin (discretize) the values in these columns. Finally we can add the new two columns to our `cols_2` list.

In [9]:
cols_2 = ['host_rt', 'host_rr', 'host_ar']

## 5. Encoding `host_is_superhost` Column
We can assign integer values to the string values we already have: `t=1, f=0`. Thankfully, this column doesn't have any missing values to deal with.

In [10]:
chicago_2['host_is_superhost'].value_counts(dropna=False)

f    3532
t    3018
Name: host_is_superhost, dtype: int64

In [11]:
super_keys = {'f': 0, 
             't': 1
             }
chicago_2['superhost'] = [super_keys[item] for item in chicago_2['host_is_superhost']]
chicago_2['superhost'].value_counts() / len(chicago_2)

0    0.539
1    0.461
Name: superhost, dtype: float64

In [12]:
chicago_2['neighborhood'] = chicago_2['neighbourhood_cleansed']
cols_2 = ['host_rt', 'host_rr', 'host_ar', 'superhost', 'neighborhood',
         'latitude', 'longitude']

Above we changed the `'neighbourhood_cleansed'` to  `'neighborhood'` and kept everything the same. There 77 unique categories in there (no missing values), so when we get to feature engineering we might have to combine minor categories to keep the number of features low. We are also not doing anything to `latitude` and `longitude`.

## 6. Dealing with `property_type` and `room_type`

In [13]:
print(chicago_2['property_type'].nunique())
chicago_2['property_type'].value_counts().head()

44


Entire apartment             2871
Private room in apartment     766
Entire condominium            648
Private room in house         487
Entire house                  393
Name: property_type, dtype: int64

In [14]:
print(chicago_2['room_type'].nunique())
chicago_2['room_type'].value_counts()

4


Entire home/apt    4524
Private room       1862
Shared room          94
Hotel room           70
Name: room_type, dtype: int64

- For `room_type`, we can just change the `Entire home/apt` to `Entire place`, because it looks and sounds better, and we can leave the rest as it is
- For `property_type`, we will have to do some work to reduce the number of categories from 44 to just a few. We really only need to know if a place is an apartment, a house, a condo or whatever, all other information that is there is given in `room_type`. There are a few ways to accomplish this:
    - We can create a dictionary with 44 keys representing each of the 44 categories and then assign each key to a value that would end up being the new category name. That would require more than 44 lines of code, so there must be a better way
    - We can create a list of word to remove from column values and in the end we would be left with words that tell us the type of the property!

In [15]:
chicago_2['room_type'] = chicago_2['room_type'].str.replace(
    'Entire home/apt', 'Entire place')
chicago_2['room_type'].value_counts()

Entire place    4524
Private room    1862
Shared room       94
Hotel room        70
Name: room_type, dtype: int64

In [16]:
remove = ['Entire', 'Private', 'Room', 'room', 'in', 'Shared', 'serviced',
          'boutique', 'guest', 'ty', 'Earth', ' ']
for word in remove:
    chicago_2['property_type'] = chicago_2['property_type'].str.replace(word, '')
chicago_2['property_type'].value_counts()

apartment          3839
condomium           957
house               951
suite               203
loft                178
townhouse           164
hotel               131
bungalow             70
bedandbreakfast      20
hostel               14
cottage               8
Boat                  4
aparthotel            3
villa                 2
farmstay              1
home/apt              1
place                 1
cab                   1
cave                  1
                      1
Name: property_type, dtype: int64

We can then combine all of the minor categories (below `bungalow`) into one:

In [17]:
minor_categories = ['bedandbreakfast', 'hostel', 'cottage', 'Boat',
                   'aparthotel', 'villa', 'farmstay', 'home/apt', 'place',
                   'cab', 'cave', '']
for word in minor_categories:
    chicago_2['property_type'] = chicago_2['property_type'].replace(word, 'other')
chicago_2['property_type'].value_counts()

apartment    3839
condomium     957
house         951
suite         203
loft          178
townhouse     164
hotel         131
bungalow       70
other          57
Name: property_type, dtype: int64

The `accommodates` column looks good!

In [26]:
chicago_2['accommodates'].value_counts(dropna=False)

2     2160
4     1350
6      778
3      514
1      434
5      406
8      314
7      155
10     152
16      86
12      77
9       47
14      29
11      29
13      11
15       7
0        1
Name: accommodates, dtype: int64

In [27]:
cols_2 = ['host_rt', 'host_rr', 'host_ar', 'superhost', 'neighborhood',
         'latitude', 'longitude', 'property_type', 'room_type', 'accommodates']

## 7. Handling `bathrooms` and `bathrooms_text`

In [28]:
chicago_2['bathrooms'].value_counts(dropna=False)

NaN    6550
Name: bathrooms, dtype: int64

In [29]:
chicago_2['bathrooms_text'].value_counts(dropna=False)

1 bath               3193
2 baths               992
1 shared bath         897
1 private bath        478
1.5 baths             190
2 shared baths        190
2.5 baths             141
3 baths               124
1.5 shared baths      105
3.5 baths              69
4 baths                40
2.5 shared baths       27
3 shared baths         27
4.5 baths              17
6 baths                 9
4 shared baths          8
5 baths                 7
NaN                     6
11 baths                5
Shared half-bath        5
0 shared baths          5
5.5 baths               3
0 baths                 3
8 baths                 2
8 shared baths          1
8.5 baths               1
Half-bath               1
7 baths                 1
12.5 baths              1
Private half-bath       1
10 baths                1
Name: bathrooms_text, dtype: int64

In [22]:
chicago_1.head()

,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month,price
0,within a few hours,100%,100%,t,Hyde Park,41.788,-87.588,Private room in condominium,Private room,1,NaN,1 shared bath,1.0,1.0,"[""Hangers"", ""Extra pillows and blankets"", ""Wif...",2,90,178,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f,2.59,$66.00
1,within an hour,100%,96%,t,South Lawndale,41.855,-87.697,Entire apartment,Entire home/apt,7,NaN,1 bath,3.0,5.0,"[""Hangers"", ""Extra pillows and blankets"", ""Wif...",2,60,395,96.0,10.0,9.0,10.0,10.0,10.0,9.0,t,2.82,$94.00
2,within an hour,100%,98%,t,West Town,41.903,-87.682,Entire apartment,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Hangers"", ""Bathtub"", ""Wifi"", ""TV"", ""Extra pi...",2,60,384,93.0,10.0,9.0,10.0,10.0,10.0,10.0,f,2.83,$61.00
3,within an hour,92%,97%,t,Lincoln Park,41.918,-87.638,Entire apartment,Entire home/apt,4,NaN,1 bath,1.0,0.0,"[""Hangers"", ""Wifi"", ""TV"", ""Smoke alarm"", ""Carb...",4,180,49,92.0,10.0,9.0,10.0,9.0,10.0,9.0,t,0.63,$67.00
4,within a few hours,90%,95%,f,Hyde Park,41.796,-87.593,Private room in apartment,Private room,2,NaN,1 shared bath,1.0,1.0,"[""Hangers"", ""Extra pillows and blankets"", ""Wif...",1,1125,44,82.0,8.0,7.0,9.0,9.0,10.0,9.0,f,0.62,$22.00
